# simple

In [ ]:
from flask import Flask, request, render_template
import pickle
app = Flask(__name__)
df=pickle.load(open('catalog.pkl','rb'))

@app.route('/', methods=['POST','GET'])    
def my_form_post():
    
    a='' 
    
    if request.method=="POST":
        a=request.form.get('name2')
        
    qs = a.lower().strip().split()

    match = lambda s: sum(min(3, s.lower().count(qp)) for qp in qs)
    pairs = []
    for pid, p in df['NAME'].items():
        score = 0.0
        score += 1.0 * match(p)
        if score > 0:
            pairs.append((score, pid))

    pairs.sort(reverse=True)
    pids = [p[1] for p in pairs]
    scores = [p[0] for p in pairs]

    d1=df.iloc[pids][['CODE','NAME']].reset_index()

    code=d1['CODE']
    name=d1['NAME']
    score=scores

    keys=['name','code','score']
    values=[name,code,score]

    newList = []
    for i in range(len(values[0])):

        tmp = {}
        for j, key in enumerate(keys):
            tmp[key] = values[j][i]

        newList.append(tmp)

    return render_template('aa2.html',data=newList)        


if __name__=='__main__':
    app.run(host='0.0.0.0', port=82, debug=False)

# compute.py

In [ ]:
import pandas as pd
import re
from pymorphy2 import MorphAnalyzer
#from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

morph = MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
#stopwords_ru = stopwords.words("russian")
stopwords_ru=['другои', 'еи', 'какои', 'мои', 'неи', 'сеичас', 'такои', 'этои','и', 'в', 'во', 'не', 'что', 'он','на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне','было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']


def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token.lower())
    if len(tokens) > 0:
        return tokens
    return None

df=pd.read_excel('catalog_v1.xlsx',sheet_name='1')

data=df['NAME'].apply(lemmatize).str.join(' ').tolist()


v = TfidfVectorizer(input='content',
                    encoding='utf-8', decode_error='replace', strip_accents='unicode',
                    lowercase=True, analyzer='word', stop_words=stopwords_ru,
                    #token_pattern=r'(?u)\b[а-яА-Я_][а-яА-Я0-9_]+\b',
                    #ngram_range=(1, 2),
                    ngram_range=(1, 1),
                    max_features=20000,
                    norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True,
                    max_df=0.1, min_df=5
                    #max_df=1, min_df=1
                   )

tfidf_vectorizer_vectors=v.fit_transform(data)

data_temp=pd.DataFrame(tfidf_vectorizer_vectors.T.todense(),index=v.get_feature_names_out())#[1].sort_values(ascending=False).head(30)

import pickle
pickle.dump(data_temp,open('catalog_vec_tfidf.pkl','wb'))
pickle.dump(v,open('model_tfidf.pkl','wb'))
pickle.dump(df,open('catalog.pkl','wb'))

In [ ]:
from flask import Flask, request, render_template
import pickle
app = Flask(__name__)

data_temp=pickle.load(open('catalog_vec_tfidf.pkl','rb'))
v=pickle.load(open('model_tfidf.pkl','rb'))
df=pickle.load(open('catalog.pkl','rb'))
    
def result_v1(search_text,data_temp=None,v=None,df=None):

    from pymorphy2 import MorphAnalyzer
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    import pandas as pd
    import re
    
    patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
    #stopwords_ru = stopwords.words("russian")
    stopwords_ru=['другои', 'еи', 'какои', 'мои', 'неи', 'сеичас', 'такои', 'этои','и', 'в', 'во', 'не', 'что', 'он','на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне','было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']
    morph = MorphAnalyzer()

    def lemmatize(doc):
        doc = re.sub(patterns, ' ', doc)
        tokens = []
        for token in doc.split():
            if token and token not in stopwords_ru:
                token = token.strip()
                token = morph.normal_forms(token)[0]

                tokens.append(token.lower())
        if len(tokens) > 0:
            return tokens
        return None


    a=[search_text]
    a=[' '.join(pd.Series(a).apply(lemmatize)[0])]
    #print(a)
    t=[cosine_similarity(np.array(v.transform(a).T.todense()).reshape(1, -1), np.array((data_temp[i])).reshape(1, -1))[0][0] for i in range(data_temp.shape[1])]
    t2=pd.DataFrame([t,df['CODE'],df['NAME']]).T.rename(columns={0:'SCORE',1:'CODE',2:'NAME'}).sort_values('SCORE',ascending=False).head(10)[['CODE','NAME','SCORE']]
    return t2['CODE'].tolist(),t2['NAME'].tolist(),t2['SCORE'].tolist()
    
@app.route('/', methods=['POST','GET'])    
def my_form_post():

    
    a='пример' 

    #newList=[{'name': 'red', 'code': 'a1', 'score': 1}, {'name': 'green', 'code': 'a5', 'score': 2}, {'name': 'blue', 'code': 'a11', 'score': 3}, {'name': 'blue2', 'code': 'a2', 'score': 5}]
    
    if request.method=="POST":
        a=request.form.get('name2')
        
    #a='молоко мячи всякое разно мебели'
    code,name,score=result_v1(a,data_temp,v,df)

    keys=['name','code','score']
    values=[name,code,score]

    newList = []
    for i in range(len(values[0])):

        tmp = {}
        for j, key in enumerate(keys):
            tmp[key] = values[j][i]

        newList.append(tmp)
    

    return render_template('aa2.html',data=newList)        


if __name__=='__main__':
    app.run(host='0.0.0.0', port=81, debug=False)